In [1]:
!pip install pytorch-tabnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 1.4 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from pytorch_tabnet.tab_model import TabNetRegressor
import torch


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
df = pd.read_excel('/content/drive/MyDrive/Nhóm 7/Đồ án môn học/DATA_TRAINING/final_dataset_before_quality_control.xlsx')
df.head()

,mssv,namsinh,gioitinh,khoa,hedt,khoahoc,dtb_toankhoa,dtb_tichluy,sotc_tichluy,diemtbhk_1,...,diemtbhk_10,diemtbhk_11,diemtbhk_12,diemtbhk_13,diemtbhk_14,diemtbhk_15,diemtbhk_16,diemtbhk_17,diemtbhk_18,diemtbhk_19
0,0000AC05XPvAibaEXe9B2tolTZ0JLoBGbkQixQS6,NaN,NaN,NaN,NaN,NaN,7.91,7.91,62.0,7.79,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0001CF3EXPvAibaEXe/xTpnJlY3K6L35F+TKUux6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,2001.0,0.0,HTTT,CQUI,14.0,8.79,8.79,129.0,8.84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00046394XPvAibaEXe+fmxcqgvribEcT4YmJhSFD,1995.0,1.0,KTMT,CQUI,8.0,7.26,7.26,147.0,5.41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0006A0BBXPvAibaEXe/lMOwHQdw54DgUkWaqwb1u,NaN,NaN,NaN,NaN,NaN,7.27,7.27,26.0,7.27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:

df = df.drop(['khoa', 'hedt', 'khoahoc', 'diemtbhk_13', 'diemtbhk_14', 'diemtbhk_15', 'diemtbhk_16', 'diemtbhk_17', 'diemtbhk_18', 'diemtbhk_19'], axis=1)
mean_nam_sinh = int(df['namsinh'].mean(skipna=True))
df['namsinh'] = df['namsinh'].fillna(mean_nam_sinh)
mode_value = df['gioitinh'].mode()[0]
df['gioitinh'] = df['gioitinh'].fillna(mode_value)
all_columns = df.columns.tolist()
exclude_columns = ['mssv', 'namsinh', 'gioitinh']
columns_to_fill = [col for col in all_columns if col not in exclude_columns]
df[columns_to_fill] = df[columns_to_fill].fillna(0)
df.head()

,mssv,namsinh,gioitinh,dtb_toankhoa,dtb_tichluy,sotc_tichluy,diemtbhk_1,diemtbhk_2,diemtbhk_3,diemtbhk_4,diemtbhk_5,diemtbhk_6,diemtbhk_7,diemtbhk_8,diemtbhk_9,diemtbhk_10,diemtbhk_11,diemtbhk_12
0,0000AC05XPvAibaEXe9B2tolTZ0JLoBGbkQixQS6,1998.0,1.0,7.91,7.91,62.0,7.79,8.28,7.56,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0
1,0001CF3EXPvAibaEXe/xTpnJlY3K6L35F+TKUux6,1998.0,1.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0
2,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,2001.0,0.0,8.79,8.79,129.0,8.84,9.00,9.11,8.75,8.54,8.74,8.41,0.0,0.0,0.0,0.0,0.0
3,00046394XPvAibaEXe+fmxcqgvribEcT4YmJhSFD,1995.0,1.0,7.26,7.26,147.0,5.41,7.07,6.36,8.32,7.86,8.09,6.92,7.8,8.2,0.0,0.0,0.0
4,0006A0BBXPvAibaEXe/lMOwHQdw54DgUkWaqwb1u,1998.0,1.0,7.27,7.27,26.0,7.27,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0


In [12]:
def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100 if np.all(y_true != 0) else np.nan
    mpe = np.mean((y_true - y_pred) / y_true) * 100 if np.all(y_true != 0) else np.nan
    r2 = r2_score(y_true, y_pred)
    return {"R2 Score": r2, "MSE": mse, "RMSE": rmse, "MAE": mae, "MAPE": mape, "MPE": mpe}

In [13]:
models = {
    "TabNetRegressor": TabNetRegressor(
        n_d=8, n_a=8, n_steps=3,
        #random_state=42
    )
}

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


In [15]:
initial_columns = ['namsinh', 'gioitinh', 'dtb_toankhoa', 'dtb_tichluy', 'sotc_tichluy', 'diemtbhk_1']

# Lặp qua từng cột làm y
results = []
current_X_columns = initial_columns.copy()

for i in range(len(initial_columns), len(df.columns) - 1):
    target_column = df.columns[i+1]

    # Xác định X và y
    X = df[current_X_columns]
    y = df[target_column]

    # Chia dữ liệu 85-15
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

    # Huấn luyện và đánh giá mô hình
    for model_name, model in models.items():
        model.fit(
            X_train.values, y_train.values.reshape(-1, 1),
            max_epochs=50,
            patience=10
        )

        # Dự đoán
        y_pred = model.predict(X_test.values)

        # Tính toán các độ đo
        metrics = calculate_metrics(y_test, y_pred)
        metrics["Model"] = model_name
        metrics["Target Column"] = target_column
        results.append(metrics)

    # Cập nhật X cho vòng tiếp theo
    current_X_columns.append(target_column)

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 11.15713|  0:00:00s
epoch 1  | loss: 2.60213 |  0:00:01s
epoch 2  | loss: 1.69183 |  0:00:01s
epoch 3  | loss: 1.59886 |  0:00:02s
epoch 4  | loss: 1.35943 |  0:00:02s
epoch 5  | loss: 1.23777 |  0:00:03s
epoch 6  | loss: 1.22347 |  0:00:04s
epoch 7  | loss: 1.141   |  0:00:05s
epoch 8  | loss: 1.1866  |  0:00:05s
epoch 9  | loss: 1.10954 |  0:00:06s
epoch 10 | loss: 1.06947 |  0:00:07s
epoch 11 | loss: 1.09229 |  0:00:08s
epoch 12 | loss: 1.07088 |  0:00:08s
epoch 13 | loss: 0.96385 |  0:00:09s
epoch 14 | loss: 0.94927 |  0:00:09s
epoch 15 | loss: 1.04282 |  0:00:10s
epoch 16 | loss: 1.06802 |  0:00:10s
epoch 17 | loss: 1.00861 |  0:00:11s
epoch 18 | loss: 1.06508 |  0:00:11s
epoch 19 | loss: 0.96141 |  0:00:12s
epoch 20 | loss: 0.90723 |  0:00:12s
epoch 21 | loss: 0.92787 |  0:00:13s
epoch 22 | loss: 0.90684 |  0:00:13s
epoch 23 | loss: 0.9099  |  0:00:14s
epoch 24 | loss: 0.93057 |  0:00:15s
epoch 25 | loss: 0.91076 |  0:00:15s
epoch 26 | loss: 0.83715 |  0:00:16s
e

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 6.67466 |  0:00:00s
epoch 1  | loss: 1.22141 |  0:00:01s
epoch 2  | loss: 0.9653  |  0:00:01s
epoch 3  | loss: 0.85951 |  0:00:02s
epoch 4  | loss: 0.77849 |  0:00:02s
epoch 5  | loss: 0.83536 |  0:00:03s
epoch 6  | loss: 0.71518 |  0:00:04s
epoch 7  | loss: 0.82252 |  0:00:04s
epoch 8  | loss: 0.67702 |  0:00:05s
epoch 9  | loss: 0.69645 |  0:00:06s
epoch 10 | loss: 0.64959 |  0:00:07s
epoch 11 | loss: 0.65494 |  0:00:08s
epoch 12 | loss: 0.6446  |  0:00:09s
epoch 13 | loss: 0.66462 |  0:00:09s
epoch 14 | loss: 0.66536 |  0:00:10s
epoch 15 | loss: 0.63257 |  0:00:10s
epoch 16 | loss: 0.61255 |  0:00:11s
epoch 17 | loss: 0.61227 |  0:00:11s
epoch 18 | loss: 0.62508 |  0:00:12s
epoch 19 | loss: 0.61293 |  0:00:12s
epoch 20 | loss: 0.59577 |  0:00:13s
epoch 21 | loss: 0.62128 |  0:00:13s
epoch 22 | loss: 0.57524 |  0:00:14s
epoch 23 | loss: 0.59524 |  0:00:14s
epoch 24 | loss: 0.58711 |  0:00:15s
epoch 25 | loss: 0.57738 |  0:00:15s
epoch 26 | loss: 0.58852 |  0:00:16s
e

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


...

epoch 0  | loss: 1.9391  |  0:00:00s
epoch 1  | loss: 1.51831 |  0:00:01s
epoch 2  | loss: 1.41519 |  0:00:02s
epoch 3  | loss: 1.35253 |  0:00:02s
epoch 4  | loss: 1.29162 |  0:00:03s
epoch 5  | loss: 1.21221 |  0:00:04s
epoch 6  | loss: 1.24167 |  0:00:04s
epoch 7  | loss: 1.17389 |  0:00:05s
epoch 8  | loss: 1.13822 |  0:00:05s
epoch 9  | loss: 1.1256  |  0:00:06s
epoch 10 | loss: 1.13438 |  0:00:06s
epoch 11 | loss: 1.16513 |  0:00:07s
epoch 12 | loss: 1.09071 |  0:00:07s
epoch 13 | loss: 1.08092 |  0:00:08s
epoch 14 | loss: 1.10171 |  0:00:08s
epoch 15 | loss: 1.08144 |  0:00:09s
epoch 16 | loss: 1.08797 |  0:00:09s
epoch 17 | loss: 1.11011 |  0:00:10s
epoch 18 | loss: 1.05578 |  0:00:10s
epoch 19 | loss: 1.13418 |  0:00:11s
epoch 20 | loss: 1.20749 |  0:00:12s
epoch 21 | loss: 1.11396 |  0:00:13s
epoch 22 | loss: 1.11164 |  0:00:13s
epoch 23 | loss: 1.09967 |  0:00:14s
epoch 24 | loss: 1.03054 |  0:00:15s
epoch 25 | loss: 1.08537 |  0:00:16s
epoch 26 | loss: 1.05163 |  0:00:16s
e

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 1.23447 |  0:00:00s
epoch 1  | loss: 0.93709 |  0:00:01s
epoch 2  | loss: 0.8517  |  0:00:01s
epoch 3  | loss: 0.75974 |  0:00:02s
epoch 4  | loss: 0.75544 |  0:00:03s
epoch 5  | loss: 0.77201 |  0:00:03s
epoch 6  | loss: 0.73657 |  0:00:04s
epoch 7  | loss: 0.70659 |  0:00:04s
epoch 8  | loss: 0.75191 |  0:00:05s
epoch 9  | loss: 0.74348 |  0:00:05s
epoch 10 | loss: 0.7205  |  0:00:06s
epoch 11 | loss: 0.6893  |  0:00:06s
epoch 12 | loss: 0.72847 |  0:00:07s
epoch 13 | loss: 0.7318  |  0:00:07s
epoch 14 | loss: 0.7187  |  0:00:08s
epoch 15 | loss: 0.69109 |  0:00:09s
epoch 16 | loss: 0.71444 |  0:00:09s
epoch 17 | loss: 0.69822 |  0:00:10s
epoch 18 | loss: 0.74019 |  0:00:11s
epoch 19 | loss: 0.73625 |  0:00:11s
epoch 20 | loss: 0.69403 |  0:00:12s
epoch 21 | loss: 0.69776 |  0:00:13s
epoch 22 | loss: 0.68584 |  0:00:14s
epoch 23 | loss: 0.69486 |  0:00:14s
epoch 24 | loss: 0.6667  |  0:00:15s
epoch 25 | loss: 0.73593 |  0:00:16s
epoch 26 | loss: 0.71204 |  0:00:16s
e

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 0.64774 |  0:00:00s
epoch 1  | loss: 0.44132 |  0:00:01s
epoch 2  | loss: 0.40454 |  0:00:01s
epoch 3  | loss: 0.40362 |  0:00:02s
epoch 4  | loss: 0.39837 |  0:00:02s
epoch 5  | loss: 0.38773 |  0:00:03s
epoch 6  | loss: 0.38857 |  0:00:03s
epoch 7  | loss: 0.37631 |  0:00:04s
epoch 8  | loss: 0.37437 |  0:00:04s
epoch 9  | loss: 0.38201 |  0:00:05s
epoch 10 | loss: 0.34513 |  0:00:05s
epoch 11 | loss: 0.38918 |  0:00:06s
epoch 12 | loss: 0.35744 |  0:00:06s
epoch 13 | loss: 0.37046 |  0:00:07s
epoch 14 | loss: 0.38072 |  0:00:07s
epoch 15 | loss: 0.37419 |  0:00:08s
epoch 16 | loss: 0.36801 |  0:00:09s
epoch 17 | loss: 0.37361 |  0:00:09s
epoch 18 | loss: 0.36511 |  0:00:10s
epoch 19 | loss: 0.36333 |  0:00:11s
epoch 20 | loss: 0.35859 |  0:00:12s
epoch 21 | loss: 0.39298 |  0:00:13s
epoch 22 | loss: 0.38339 |  0:00:13s
epoch 23 | loss: 0.38029 |  0:00:14s
epoch 24 | loss: 0.34774 |  0:00:15s
epoch 25 | loss: 0.37376 |  0:00:15s
epoch 26 | loss: 0.36839 |  0:00:16s
e

In [16]:
# Tạo DataFrame kết quả
results_df = pd.DataFrame(results)
results_df.to_csv("tabnet_model_results.csv", index=False)

print(results_df)

     R2 Score         MSE       RMSE        MAE  MAPE  MPE            Model  \
0   -0.524790   19.071490   4.367092   3.204596   NaN  NaN  TabNetRegressor   
1    0.510229    6.437955   2.537313   1.733174   NaN  NaN  TabNetRegressor   
2    0.635585    4.808301   2.192784   1.321450   NaN  NaN  TabNetRegressor   
3    0.204238   10.665192   3.265761   2.472858   NaN  NaN  TabNetRegressor   
4    0.586197    5.265857   2.294745   1.158629   NaN  NaN  TabNetRegressor   
5    0.773076    2.938178   1.714111   1.050980   NaN  NaN  TabNetRegressor   
6  -11.911594  133.619783  11.559402  11.186475   NaN  NaN  TabNetRegressor   
7    0.347239    4.770190   2.184076   1.087702   NaN  NaN  TabNetRegressor   
8  -19.361397   42.094025   6.487991   3.185069   NaN  NaN  TabNetRegressor   
9    0.377132    0.606815   0.778983   0.231249   NaN  NaN  TabNetRegressor   
10  -0.091587    0.367749   0.606423   0.150642   NaN  NaN  TabNetRegressor   

   Target Column  
0     diemtbhk_2  
1     diemtbh

In [17]:
print(results_df['R2 Score'].mean())
print(results_df['MSE'].mean())
print(results_df['RMSE'].mean())
print(results_df['MAE'].mean())

-2.5868793631430473
20.967775784081482
3.4535164661840083
2.4348022524756114
